EUREKA

i can use data algo project code to make new templates off the signal code and just change signal inputs
also, generate an email each time
also, generate the code for stocks, etf, and forex and futures!

simply alter code for oanda and use this template, perfect it for a backtest and simply convert to forward test when ready to 
try it in a live market but fake account, when ready for real account, simply add oanda account info etc and key

continue to develop the code so it runs cleanly and generate solid seaborn graphics, add a scheduler for forward test and live deployment. 

oct 3, 2023 7 58 pm 
good work on a late start 

In [61]:
import oandapyV20
import oandapyV20.endpoints.instruments as instruments
import pandas as pd
from datetime import datetime, timedelta
import time
import oandapyV20
import oandapyV20.endpoints.instruments as instruments
import numpy as np
import matplotlib.pyplot as plt
import warnings 
# Ignore all warnings
warnings.filterwarnings('ignore')

In [62]:

# OANDA API credentials
api_token = '4774a622bc314440290f530efcc46453-eb96728988593ba944d5285fbcf8786a'
account_id = '101-001-8028197-001'

# OANDA API connection
api = oandapyV20.API(access_token=api_token)

In [63]:
# Download Data for Backtest 
client = oandapyV20.API(access_token=api_token)

# Get historical data for EURUSD
instrument = "EUR_USD"
start_date = "2020-06-01"
end_date = "2023-10-03"
params = {"from": start_date, "to": end_date, "granularity": "D"}
r = instruments.InstrumentsCandles(instrument=instrument, params=params)
candles = client.request(r)["candles"]

# Create a DataFrame from the historical data
df = pd.DataFrame([{"time": c["time"], "open": c["mid"]["o"], "high": c["mid"]["h"], "low": c["mid"]["l"], "close": c["mid"]["c"]} for c in candles])
df["time"] = pd.to_datetime(df["time"])

display(df)

,time,open,high,low,close
0,2020-05-31 21:00:00+00:00,1.11053,1.11542,1.11004,1.11345
1,2020-06-01 21:00:00+00:00,1.11360,1.11962,1.11154,1.11692
2,2020-06-02 21:00:00+00:00,1.11692,1.12580,1.11669,1.12332
3,2020-06-03 21:00:00+00:00,1.12340,1.13626,1.11946,1.13364
4,2020-06-04 21:00:00+00:00,1.13378,1.13839,1.12784,1.12916
...,...,...,...,...,...
864,2023-09-26 21:00:00+00:00,1.05698,1.05743,1.04880,1.05026
865,2023-09-27 21:00:00+00:00,1.05074,1.05790,1.04911,1.05664
866,2023-09-28 21:00:00+00:00,1.05616,1.06171,1.05581,1.05744
867,2023-10-01 21:00:00+00:00,1.05720,1.05919,1.04779,1.04780


In [64]:
# Calculate 10-day moving average of the open prices
df['MA10'] = df['open'].rolling(window=10).mean()

# Convert Data types for easy integration
df['MA10'] = df['MA10'].astype(float)
df['open'] = df['open'].astype(float)
df['high'] = df['high'].astype(float)
df['low'] = df['low'].astype(float)
df['close'] = df['close'].astype(float)

# Create a 'Signal' column indicating when the short moving average crosses the long.
df['Signal'] = np.where(df['open'] > df['MA10'], 1.0, 0.0)

# Create a new column named "Order", buy the day after (diff) when signal is generated, buy on the open, sell at the close! 
df["StratOrder"] = ""
StratOrder = df['StratOrder']
df.loc[df["Signal"] == 1.0, "StratOrder"] = "Buy"

df['Signal'] = 0  # 0 represents no signal
df.loc[df['open'] < df['MA10'], 'Signal'] = -1  # Sell signal
df.loc[df['open'] > df['MA10'], 'Signal'] = 1   # Buy signal

# Drop null values 
df = df.dropna()

In [65]:
display(df)

,time,open,high,low,close,MA10,Signal,StratOrder
9,2020-06-11 21:00:00+00:00,1.12956,1.13406,1.12126,1.12554,1.125838,1,Buy
10,2020-06-14 21:00:00+00:00,1.12344,1.13327,1.12266,1.13228,1.127129,-1,
11,2020-06-15 21:00:00+00:00,1.13246,1.13534,1.12276,1.12632,1.129015,1,Buy
12,2020-06-16 21:00:00+00:00,1.12636,1.12942,1.12070,1.12436,1.129959,-1,
13,2020-06-17 21:00:00+00:00,1.12427,1.12615,1.11855,1.12049,1.130046,-1,
...,...,...,...,...,...,...,...,...
864,2023-09-26 21:00:00+00:00,1.05698,1.05743,1.04880,1.05026,1.065352,-1,
865,2023-09-27 21:00:00+00:00,1.05074,1.05790,1.04911,1.05664,1.063134,-1,
866,2023-09-28 21:00:00+00:00,1.05616,1.06171,1.05581,1.05744,1.062362,-1,
867,2023-10-01 21:00:00+00:00,1.05720,1.05919,1.04779,1.04780,1.061455,-1,


In [66]:
# Filtering the DataFrame
buy_df = df[df['StratOrder'].isin(['Buy'])]
# Displaying the result
display(buy_df)

,time,open,high,low,close,MA10,Signal,StratOrder
9,2020-06-11 21:00:00+00:00,1.12956,1.13406,1.12126,1.12554,1.125838,1,Buy
11,2020-06-15 21:00:00+00:00,1.13246,1.13534,1.12276,1.12632,1.129015,1,Buy
17,2020-06-23 21:00:00+00:00,1.13095,1.13260,1.12483,1.12507,1.126951,1,Buy
21,2020-06-29 21:00:00+00:00,1.12401,1.12620,1.11911,1.12331,1.123950,1,Buy
23,2020-07-01 21:00:00+00:00,1.12514,1.13026,1.12234,1.12394,1.123744,1,Buy
...,...,...,...,...,...,...,...,...
827,2023-08-06 21:00:00+00:00,1.10175,1.10206,1.09654,1.10027,1.100802,1,Buy
828,2023-08-07 21:00:00+00:00,1.10033,1.10113,1.09290,1.09565,1.100181,1,Buy
831,2023-08-10 21:00:00+00:00,1.09821,1.10051,1.09430,1.09468,1.098186,1,Buy
844,2023-08-29 21:00:00+00:00,1.08754,1.09456,1.08552,1.09234,1.085353,1,Buy


In [67]:

# Find values in 'Open' and 'Close' columns in the row after 'Buy' in 'Strat Order'
buy_indices = df[df['StratOrder'] == 'Buy'].index

# Finds the next day to operate buy signal on!  Eureka! 
next_row_indices = buy_indices + 1
# Make the dataframe for next rows so we can calculate balance etc afterwards
nextday_df = df.loc[next_row_indices, ['time','open', 'close','high','low','MA10',]]
ndf = nextday_df
# Display the result DataFrame
display(ndf)

,time,open,close,high,low,MA10
10,2020-06-14 21:00:00+00:00,1.12344,1.13228,1.13327,1.12266,1.127129
12,2020-06-16 21:00:00+00:00,1.12636,1.12436,1.12942,1.12070,1.129959
18,2020-06-24 21:00:00+00:00,1.12528,1.12180,1.12598,1.11906,1.125728
22,2020-06-30 21:00:00+00:00,1.12343,1.12514,1.12751,1.11848,1.123657
24,2020-07-02 21:00:00+00:00,1.12394,1.12479,1.12518,1.12194,1.124089
...,...,...,...,...,...,...
828,2023-08-07 21:00:00+00:00,1.10033,1.09565,1.10113,1.09290,1.100181
829,2023-08-08 21:00:00+00:00,1.09539,1.09736,1.09956,1.09518,1.099192
832,2023-08-13 21:00:00+00:00,1.09532,1.09072,1.09604,1.08748,1.097523
845,2023-08-30 21:00:00+00:00,1.09262,1.08434,1.09398,1.08351,1.085712


Now, let's add more columns to our filtered dataframe with all subsequent rows on which the transactions occur.

In [68]:
# Create Delta column to gauge intraday change in price 
ndf['Delta']= ndf['close']-ndf['open']

# Create Range column to track distance between low and high 
ndf['Range']= ndf['high']-ndf['low']

# Create Unit Tradesize Column which is in units instead of dollars, longer explanation detailing margin, unit size, leverage etc. 
ndf['Unit Size']= 50000

# Create Tradesize 
ndf['Tradesize'] = ndf['Unit Size'] * ndf['close']

# Create Margin Used on particular trade 
ndf['Margin Used'] = .02 * ndf['Tradesize']

# Create the Strategy profit column using delta (intraday change) and unit size
ndf['StratProfit']= ndf['Delta'] * ndf['Unit Size']

# Define and Calculate Balance and Strat Initial Balance (the price at the open of first day trade)
SIB = 10000
cum_profit = ndf['StratProfit'].cumsum()
ndf['Strat_Balance'] = cum_profit  + SIB 
Strat_Balance = ndf['Strat_Balance']



# Win Loss Column added
def is_profitable(row):
    if row['StratProfit'] > 0:
        return 'Win'
    else:
        return 'Loss'

ndf['WinLoss'] = ndf.apply(is_profitable, axis=1)

# Add 'WIN' column to store positive profits
def get_win_amount(row):
    if row['StratProfit'] > 0:
        return row['StratProfit']
    else:
        return 0.00

ndf['Win'] = ndf.apply(get_win_amount, axis=1)

# Add 'Loss' column to store negative profits
def get_loss_amount(row):
    if row['StratProfit'] < 0:
        return row['StratProfit']
    else:
        return 0.00

ndf['Loss'] = ndf.apply(get_loss_amount, axis=1)

# Strategy Returns Column 
ndf['Strat_Returns_pct'] =((Strat_Balance/SIB) -1)*100
Strat_Returns = ndf['Strat_Returns_pct']

# Calculate the strat drawdown
stratdrawdown = ndf["Strat_Balance"].max() - ndf["Strat_Balance"]
ndf['StratDrawdown'] = stratdrawdown

# Buy and Hold as a Benchmark 
# Create bhBalance from shares * close 

bhBalance = 50000 * ndf['close']
ndf['bhBalance']= bhBalance

# Calculate the bh drawdown
drawdown = ndf["bhBalance"].max() - ndf["bhBalance"]
ndf['bhDrawdown'] = drawdown



# Buy and Hold Strategy Pct Return and Cumulative Strategy Pct Return 

# Define initial balance and strategy balance
initial_balance = 10000  # Example initial balance
bhbalance = ndf['bhBalance']    # Example strategy balance

# Calculate bnh return
bhreturnpct= ((bhbalance - initial_balance) / initial_balance) 


# Return is cumulative for bh strategy (buy and hold strategy)
ndf['bhreturnpct']=bhreturnpct 

display(ndf)



,time,open,close,high,low,MA10,Delta,Range,Unit Size,Tradesize,...,StratProfit,Strat_Balance,WinLoss,Win,Loss,Strat_Returns_pct,StratDrawdown,bhBalance,bhDrawdown,bhreturnpct
10,2020-06-14 21:00:00+00:00,1.12344,1.13228,1.13327,1.12266,1.127129,0.00884,0.01061,50000,56614.0,...,442.0,10442.0,Win,442.0,0.0,4.420,2900.5,56614.0,5022.0,4.66140
12,2020-06-16 21:00:00+00:00,1.12636,1.12436,1.12942,1.12070,1.129959,-0.00200,0.00872,50000,56218.0,...,-100.0,10342.0,Loss,0.0,-100.0,3.420,3000.5,56218.0,5418.0,4.62180
18,2020-06-24 21:00:00+00:00,1.12528,1.12180,1.12598,1.11906,1.125728,-0.00348,0.00692,50000,56090.0,...,-174.0,10168.0,Loss,0.0,-174.0,1.680,3174.5,56090.0,5546.0,4.60900
22,2020-06-30 21:00:00+00:00,1.12343,1.12514,1.12751,1.11848,1.123657,0.00171,0.00903,50000,56257.0,...,85.5,10253.5,Win,85.5,0.0,2.535,3089.0,56257.0,5379.0,4.62570
24,2020-07-02 21:00:00+00:00,1.12394,1.12479,1.12518,1.12194,1.124089,0.00085,0.00324,50000,56239.5,...,42.5,10296.0,Win,42.5,0.0,2.960,3046.5,56239.5,5396.5,4.62395
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
828,2023-08-07 21:00:00+00:00,1.10033,1.09565,1.10113,1.09290,1.100181,-0.00468,0.00823,50000,54782.5,...,-234.0,8752.5,Loss,0.0,-234.0,-12.475,4590.0,54782.5,6853.5,4.47825
829,2023-08-08 21:00:00+00:00,1.09539,1.09736,1.09956,1.09518,1.099192,0.00197,0.00438,50000,54868.0,...,98.5,8851.0,Win,98.5,0.0,-11.490,4491.5,54868.0,6768.0,4.48680
832,2023-08-13 21:00:00+00:00,1.09532,1.09072,1.09604,1.08748,1.097523,-0.00460,0.00856,50000,54536.0,...,-230.0,8621.0,Loss,0.0,-230.0,-13.790,4721.5,54536.0,7100.0,4.45360
845,2023-08-30 21:00:00+00:00,1.09262,1.08434,1.09398,1.08351,1.085712,-0.00828,0.01047,50000,54217.0,...,-414.0,8207.0,Loss,0.0,-414.0,-17.930,5135.5,54217.0,7419.0,4.42170


We have constructed all of our data and findings for our backtest. 

We can use this data and dataframe to answer the following questions:

1.  What were the results of the strategy?
2.  What was the drawdown?
3.  How did the benchmark perform?
4.  What were the risk metrics? 
5.  What were the average profits? Losses?
6.  What was the win/loss count?
7.  What is the average daily range? 
8.  What is the average daily change?
9.  What is the rate of return on the strategy?
10. S